In [13]:
%reset -f
!pip install nltk
!pip install xgboost
!pip install imblearn
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
#from wordcloud import WordCloud
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer



#embedding_2=pd.read_csv('../xwang3306/aliba_embedding_10p.csv')

In [ ]:

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

# Load your data (replace with your actual data loading)
# Assuming your data is in a CSV file named 'data.csv'
import pandas as pd


data_size = '10p'
# data_size = 'full'
version = 'v1'

data_10P = pd.read_csv('../data/neiss_10p_sample.csv') if data_size == '10p' else pd.read_csv('../data/consolidated_cleaned_neiss_2014_2023.csv')

# data_10P = pd.read_csv('../data/neiss_10p_sample.csv')
if data_size == '10p':
    data_10P = data_10P.drop(columns=['Unnamed: 0'])

print("data_10p", data_10P.shape)

embedding=pd.read_csv(f'../data/gist_embedding_{data_size}_{version}.csv')
print("embedding", embedding.shape)
# print("new_columns", new_columns.shape)
print(embedding.head(10))

[nltk_data] Downloading package punkt to /home/eric/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/eric/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/eric/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/eric/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


data_10p (352052, 28)
embedding (352052, 385)
   CPSC_Case_Number         0         1         2         3         4  \
0         221032332 -0.041426 -0.019759  0.047550 -0.016496 -0.010686   
1         181109464 -0.017625 -0.034500  0.065491  0.020524  0.021781   
2         210103105 -0.056190 -0.016024  0.018050  0.011191 -0.066218   
3         161157997 -0.039718 -0.044306  0.027221  0.031631 -0.025631   
4         181107411 -0.026992 -0.003040  0.037108 -0.065492 -0.010564   
5         200134239  0.000362  0.013022  0.013661 -0.009646 -0.016427   
6         140951498 -0.044273  0.000546  0.037143 -0.014956 -0.030164   
7         221017396 -0.032237 -0.021063  0.026084 -0.042346 -0.011767   
8         200645623 -0.011380 -0.037507  0.041981 -0.064022 -0.016811   
9         141040420  0.019674 -0.015743  0.015878 -0.029989  0.005483   

          5         6         7         8  ...       374       375       376  \
0  0.015076  0.113167  0.052858 -0.000614  ...  0.034040 -0.036666  0.

In [15]:
# data=data_10P.merge(new_columns,how='inner',on='CPSC_Case_Number').merge(embedding,how='inner',on='CPSC_Case_Number').reset_index(drop=True)
data_10P = data_10P[['CPSC_Case_Number', 'Age', 'Sex', 'Body_Part', 'Location', 'Product_1', 'Disposition']]
data=data_10P.merge(embedding,how='inner',on='CPSC_Case_Number').reset_index(drop=True)
del data_10P
del embedding
# data_10P.head(10)
# embedding.head(10)
print(data.shape)
data.head(10)

(352052, 391)


,CPSC_Case_Number,Age,Sex,Body_Part,Location,Product_1,Disposition,0,1,2,...,374,375,376,377,378,379,380,381,382,383
0,221032332,14,1,34,0,1205,1,-0.041426,-0.019759,0.047550,...,0.034040,-0.036666,0.008797,0.024984,-0.048995,0.002708,-0.020119,-0.058474,-0.022191,0.038710
1,181109464,28,1,79,1,1141,1,-0.017625,-0.034500,0.065491,...,0.024239,-0.000142,0.007269,-0.015545,-0.040856,-0.042023,-0.050380,0.008760,-0.002266,0.043031
2,210103105,35,1,30,9,5033,1,-0.056190,-0.016024,0.018050,...,0.060739,0.014632,0.022998,-0.055498,-0.040177,0.024382,-0.086975,-0.045614,-0.022894,-0.013169
3,161157997,214,2,76,0,1842,1,-0.039718,-0.044306,0.027221,...,0.027231,-0.016303,-0.013848,-0.046165,-0.034185,0.006101,-0.041465,-0.041030,0.040551,0.013185
4,181107411,4,1,92,1,5020,1,-0.026992,-0.003040,0.037108,...,0.019272,-0.061428,0.017961,0.035712,0.021475,0.038897,-0.028660,0.035182,-0.031062,-0.006973
5,200134239,207,1,82,1,1682,1,0.000362,0.013022,0.013661,...,0.015271,-0.032191,-0.001334,0.017044,-0.005507,-0.011875,-0.101678,0.020554,0.008349,0.024304
6,140951498,12,1,82,8,1200,1,-0.044273,0.000546,0.037143,...,0.073003,-0.003709,0.004067,-0.014014,-0.029929,0.032725,-0.047167,-0.049677,-0.029124,0.038116
7,221017396,44,2,79,0,557,1,-0.032237,-0.021063,0.026084,...,0.016179,-0.031867,-0.007946,-0.036613,-0.030827,0.035943,-0.010132,-0.029518,-0.000366,0.040291
8,200645623,28,1,30,0,1333,1,-0.011380,-0.037507,0.041981,...,0.068889,-0.028480,-0.011834,0.026125,-0.003834,0.022431,-0.116032,-0.013971,-0.105048,0.038655
9,141040420,16,2,79,9,1266,1,0.019674,-0.015743,0.015878,...,0.000669,0.020707,-0.023070,0.032140,-0.013655,0.035791,-0.008253,-0.096477,0.032906,0.039108


In [16]:
print(list(data.columns))

['CPSC_Case_Number', 'Age', 'Sex', 'Body_Part', 'Location', 'Product_1', 'Disposition', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', 

In [17]:
import numpy as np
data['Disposition_recode']=np.nan
data.loc[((data['Disposition']==1)), 'Disposition_recode'] = 0
data.loc[((data['Disposition']==2)), 'Disposition_recode'] = 1
data.loc[((data['Disposition']==4)), 'Disposition_recode'] = 2
data.loc[((data['Disposition']==5)), 'Disposition_recode'] = 3
data.loc[((data['Disposition']==8)), 'Disposition_recode'] = 4
data=data[data['Disposition_recode'].notna()]

data['Disposition_recode_2']=0
data.loc[((data['Disposition_recode']>0)), 'Disposition_recode_2'] = 1

In [18]:
data=data[(data['Body_Part']!=0) & (data['Body_Part']!=84) & (data['Body_Part']!=85) & (data['Body_Part']!=86) & (data['Body_Part']!=87)]
data['Disposition_recode'].value_counts()
data['Disposition_recode_2'].value_counts() 

Disposition_recode_2
0    296228
1     32874
Name: count, dtype: int64

In [19]:
bdpt_dict={}
bdpt_dict[0]='INTERNAL'
bdpt_dict[30]='SHOULDER'
bdpt_dict[31]='UPPERTRUNK'
bdpt_dict[32]='ELBOW'
bdpt_dict[33]='LOWERARM'
bdpt_dict[34]='WRIST'
bdpt_dict[35]='KNEE'
bdpt_dict[36]='LOWERLEG'
bdpt_dict[37]='ANKLE'
bdpt_dict[38]='PUBICREGION'
bdpt_dict[75]='HEAD'
bdpt_dict[76]='FACE'
bdpt_dict[77]='EYEBALL'
bdpt_dict[78]='UPPERTRUNK(OLD)'
bdpt_dict[79]='LOWERTRUNK'
bdpt_dict[80]='UPPERARM'
bdpt_dict[81]='UPPERLEG'
bdpt_dict[82]='HAND'
bdpt_dict[83]='FOOT'
bdpt_dict[84]='25-50% OF BODY'
bdpt_dict[85]='ALLPARTSBODY'
bdpt_dict[86]='OTHER(OLD)'
bdpt_dict[87]='NOTSTATED/UNK'
bdpt_dict[88]='MOUTH'
bdpt_dict[89]='NECK'
bdpt_dict[90]='LOWERARM(OLD)'
bdpt_dict[91]='LOWERLEG(OLD)'
bdpt_dict[92]='FINGER'
bdpt_dict[93]='TOE'
bdpt_dict[94]='EAR'


In [20]:
data['body_string']=data['Body_Part'].map(bdpt_dict)
# data['Narrative_LLM']=data["activity_at_injury"].astype(str) + ' '+data["injury_mechanism"].astype(str)+ ' ' + data["object_involved"].astype(str)

In [21]:

total_rows, n_columns = data.shape

test_size = int(total_rows * 0.2)
train_size = total_rows - test_size

print(f"Original dataset shape: ({total_rows}, {n_columns})")
print(f"Train set size: {train_size} rows")
print(f"Test set size: {test_size} rows")
data_sample = data.sample(frac=1,random_state=42).reset_index(drop=True)

data_ready = data_sample.tail(train_size).reset_index(drop=True)
data_ready_test = data_sample.head(test_size).reset_index(drop=True)
del data

# data_test=data_core.head(21000).reset_index(drop=True)
# data_fit=data_core.tail(191347).reset_index(drop=True)
# df_bad=data_fit[data_fit['Disposition_recode_2']==1]
# df_good=data_fit[data_fit['Disposition_recode_2']==0]
# data_good_sample=df_good.sample(frac=0.2,random_state=42).reset_index(drop=True)
# data_bad_sample=df_bad.sample(frac=0.8,random_state=42).reset_index(drop=True)
# data_core_sample=pd.concat([data_good_sample,data_bad_sample]).reset_index(drop=True)
# data_core_sample=data_core_sample.sample(frac=1,random_state=42).reset_index(drop=True)
# data_core_sample.head()

# data_ready = data_core_sample
# data_ready_test = data_test
data_ready.columns

Original dataset shape: (329102, 394)
Train set size: 263282 rows
Test set size: 65820 rows


Index(['CPSC_Case_Number', 'Age', 'Sex', 'Body_Part', 'Location', 'Product_1',
       'Disposition', '0', '1', '2',
       ...
       '377', '378', '379', '380', '381', '382', '383', 'Disposition_recode',
       'Disposition_recode_2', 'body_string'],
      dtype='object', length=394)

In [22]:
drop_list=[
 'CPSC_Case_Number',
 'Disposition',
 'Disposition_recode',
 'Disposition_recode_2',
 'body_string',
]

drop_list_test = drop_list

In [23]:
X = data_ready.drop(drop_list, axis=1) 
y = data_ready['Disposition_recode_2']

print(list(X.columns))

['Age', 'Sex', 'Body_Part', 'Location', 'Product_1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', 

In [ ]:
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_auc_score
from scipy.stats import mode
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import numpy as np
from sklearn.neural_network import MLPClassifier

# Separate features (X) and target variable (y)
X = data_ready.drop(drop_list, axis=1) 
y = data_ready['Disposition_recode_2']

# Encode target variable if it's categorical
le = LabelEncoder()
y = le.fit_transform(y)

# Split data into training and testing sets
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)

X_test=data_ready_test.drop(drop_list_test, axis=1)
y_test=data_ready_test['Disposition_recode_2']
y_test = le.fit_transform(y_test)

# Create a scaler for numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include=['number']))

with open(f'X_scaler_{data_size}_{version}.pkl', 'wb') as file:
    pickle.dump(scaler, file)

X_test_scaled = scaler.transform(X_test.select_dtypes(include=['number']))

# Convert scaled features back to DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.select_dtypes(include=['number']).columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.select_dtypes(include=['number']).columns)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled_df, y_train)

# Combine scaled numerical features with categorical features
X_train_final = X_resampled.copy()
X_test_final = X_test_scaled_df.copy()
y_train_final=y_resampled.copy()


# Define models and their parameter grids for hyperparameter tuning
models = {
    #'KNN': (KNeighborsClassifier(), {'knn__n_neighbors': [3, 5, 7]}),
    #'Random_Forest': (RandomForestClassifier(), {'random_forest__n_estimators': [100, 200], 'random_forest__max_depth': [4, 8]}),
    'XGBoost': (XGBClassifier(objective='binary:logistic'), 
           {'xgboost__learning_rate': [0.1, 0.01, 0.001],            # Step size shrinkage to prevent overfitting
            'xgboost__max_depth': [3, 5, 7],                         # Maximum depth of each tree
            #'xgboost__n_estimators': [100, 200, 300],                # Number of boosting rounds
            #'xgboost__min_child_weight': [1, 3, 5],                  # Minimum sum of instance weight needed in a child
            #'xgboost__subsample': [0.6, 0.8,0.9],                   # Subsample ratio of the training instances
            #'xgboost__colsample_bytree': [0.6, 0.8, 0.9],            # Subsample ratio of columns when constructing each tree
            #'xgboost__gamma': [0.1, 0.5, 1,3],                     # Minimum loss reduction required for a split
            #'xgboost__reg_alpha': [0.01, 0.1,0.5,1],                    # L1 regularization term on weights
            #'xgboost__reg_lambda': [1, 1.5, 2],                      # L2 regularization term on weights
            'xgboost__scale_pos_weight': [1, 5, 10]
            }),
    #'Logistic_Regression': (LogisticRegression(solver='liblinear'), {'logistic_regression__C': [0.1, 1, 10]}),
    #'SVM': (SVC(probability=True), {'svm__C': [0.1, 1, 10], 'svm__kernel': ['linear', 'rbf']}),
    #'NN': (MLPClassifier(max_iter=1000), {'nn__hidden_layer_sizes': [(10,), (50,), (100,)], 'nn__activation': ['relu', 'tanh']}),
}

# Initialize list to store predictions from each model

ensemble_predictions = []
ensemble_probabilities = []

for model_name, (model, param_grid) in models.items():
    print(f"Training {model_name}...")

    # Create a pipeline for preprocessing and model training
    pipeline = Pipeline([
        (model_name.lower(), model)
    ])

    # Perform hyperparameter tuning using GridSearchCV with AUC score
    grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='precision',n_jobs=8)
    grid_search.fit(X_train_final, y_train_final)

    # Get the best model from the grid search
    best_model = grid_search.best_estimator_

    # Make predictions and predict probabilities on the test set
    y_pred = best_model.predict(X_test_final)
    y_prob = best_model.predict_proba(X_test_final)[:, 1]  # Probability for the positive class
    ensemble_predictions.append(y_pred)
    ensemble_probabilities.append(y_prob)

    # Evaluate the model using accuracy, F1-score, and AUC
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} F1 Score: {f1}")
    print(f"{model_name} AUC: {auc}")
    print(classification_report(y_test, y_pred))

# # Implement ensemble method (majority voting for predictions, average for probabilities)
# ensemble_predictions = np.array(ensemble_predictions)
# ensemble_pred_final = mode(ensemble_predictions, axis=0)[0].flatten()

# # Average the probabilities for the ensemble AUC
# ensemble_probabilities = np.mean(ensemble_probabilities, axis=0)

# # Evaluate the ensemble model using accuracy, F1-score, and AUC
# ensemble_accuracy = accuracy_score(y_test, ensemble_pred_final)
# ensemble_f1 = f1_score(y_test, ensemble_pred_final)
# ensemble_auc = roc_auc_score(y_test, ensemble_probabilities)
# print("Ensemble Accuracy:", ensemble_accuracy)
# print("Ensemble F1 Score:", ensemble_f1)
# print("Ensemble AUC:", ensemble_auc)
# print(classification_report(y_test, ensemble_pred_final))

with open(f'xgboost_embedding_{data_size}_{version}.pkl', 'wb') as file:
    pickle.dump(best_model, file)

Training XGBoost...


Exception ignored in: <function _releaseLock at 0x7a47685f5940>
Traceback (most recent call last):
  File "/home/eric/miniconda3/envs/neiss/lib/python3.12/logging/__init__.py", line 243, in _releaseLock
    def _releaseLock():
    
KeyboardInterrupt: 


In [ ]:
X_test_final.head(5)


,Age,Sex,Body_Part,Location,Product_1,0,1,2,3,4,...,374,375,376,377,378,379,380,381,382,383
0,-0.380845,-0.919097,0.797135,2.028643,0.830206,-0.587897,-0.809677,-0.002741,0.429119,-0.628006,...,-0.186046,-0.222702,0.905786,-0.695474,0.674217,-0.574596,0.007002,-1.302847,-0.978701,-1.300178
1,-0.255708,-0.919097,0.624982,0.479407,2.139031,-1.090120,-0.228398,-0.269642,-0.213726,-0.653717,...,-1.247641,0.831428,-0.210775,0.417983,-1.319875,1.101192,-0.760271,1.106522,0.471580,-1.654339
2,-0.735400,1.087442,-1.182620,-0.450134,-0.639364,-0.238149,0.045391,0.849737,-0.784145,-0.576590,...,-1.766205,0.418259,2.197462,-0.133819,-0.422083,0.699439,-0.119825,-0.025478,-0.751724,-0.013726
3,-0.818824,1.087442,-1.354773,2.028643,-0.661485,0.544034,-1.295379,1.955023,-0.717170,-0.858801,...,1.626604,0.096559,-0.308833,0.374799,-0.522339,-0.520333,-1.468355,1.667375,-2.012132,2.198260
4,-0.818824,1.087442,0.495868,-0.450134,-0.188096,-0.224956,0.679287,1.140302,0.655224,0.485271,...,1.136586,-0.255621,0.844490,-0.623350,1.510537,-0.916066,1.234783,1.747175,-1.182524,1.181839


In [ ]:
scaled_body_parts = X_test_final['Body_Part'].unique()
scaled_body_parts.sort()
scaled_body_parts

array([-1.48388719, -1.44084903, -1.39781088, -1.35477273, -1.31173458,
       -1.26869643, -1.22565828, -1.18262013, -1.13958198,  0.45282959,
        0.49586774,  0.53890589,  0.62498219,  0.66802035,  0.7110585 ,
        0.75409665,  0.7971348 ,  1.01232555,  1.0553637 ,  1.18447815,
        1.2275163 ,  1.27055445])

In [ ]:
for index, row in X_test_final[:10].iterrows():
    prob = []
    for body_part in scaled_body_parts:
        current_sample = row.copy()
        current_sample['Body_Part'] = body_part
        # print(current_sample)
        prob.append(best_model.predict_proba([current_sample])[0][0])
    prob.sort()
    print(prob[0], prob[-1]) # min max

0.68951863 0.9617856
0.16811895 0.7241533
0.76339394 0.97560763
0.7927586 0.9804928
0.72279125 0.9792716
0.83525884 0.98364496
0.85460955 0.99233586
0.7927562 0.9863167
0.971513 0.9867446
0.8203353 0.9721784


In [ ]:
%reset -f